<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>
Figure 1.1.a. Jupyter environment + Python notebooks

### Digital Earth Pacific Notebook 1 RVI testing J.b 2025



In [1]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio.features
import geopandas as gpd

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer as pc

# Other tools
from scipy.ndimage import uniform_filter, variance
from PIL import Image
import odc
from odc.stac import stac_load

### Define the analysis region and load the data

In [2]:
#geojson
gdf = gpd.read_file("training-data/Malakula.geojson")

gdf = gdf.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = gdf.total_bounds
bbox = [min_lon, min_lat, max_lon, max_lat]
print(bbox)

[167.01390995934008, -16.600698523048678, 167.86259603502924, -15.843398137832736]


In [3]:
time_of_interest = ('2020-08-01', '2020-08-31')


In [4]:
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [5]:
# We will only load a specific orbit path (descending or ascending)

search = catalog.search(
    collections=["sentinel-1-rtc"], 
    bbox=bbox, 
    datetime=time_of_interest,
    query={"sat:orbit_state": {"eq": "descending"}}
)

In [6]:
items = list(search.get_all_items()) # This produces a list of scene IDs

In [7]:
print('This is the number of scenes that touch our region:',len(items))

This is the number of scenes that touch our region: 2


Next, we'll load the data into an XARRAY and then "clip" the data to only the pixels within our region (bounding box). We will only keep the desired bands (VV, VH). We have changed the projection to epsg=4326 which is standard latitude-longitude in degrees and we have specified the spatial resolution of each pixel to be 10-meters, which is the baseline accuracy for this data. 

In [ ]:
# Define the pixel resolution for the final product
# Define the scale according to our selected crs, so we will use degrees
# The best resolution for Sentinel-1 is 10-meters
# Use 100-meters for rapid review of the datasets

resolution = 20  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for crs=4326 